In [30]:
import torch
from torch import Tensor
import numpy as np

# Auxiliary Classes

In [172]:
class Weight:
    """
    Initializes the synaptic weights
    
    input_s:
        layer_nb    :  the number of layer
        dim_in      :  the input_ dimension of fully connected layer
        dim_out     :  the output_ dimension of fully connected layer
        
    returns:
        Nothing
    """
    
    def __init__(self, dim_in, dim_out, s=[]):
        self.bias = Tensor(dim_out,1).fill_(0)      # bias of each layer
        self.w = Tensor(dim_out,dim_in).normal_()   # w of each layer
        self.s = s                                  # s after each layer

# it is a module in the net. it calls some specific parameter...   


class Linear:
    """
    An class that contains objects which only store layar's in/out connections dimension
    
    input_s:
        dim_in      :  the input_ dimension of fully connected layer
        dim_out     :  the output_ dimension of fully connected layer
        
    returns:
        Nothing 
    """
    
    def __init__(self,dim_in, dim_out):
        self.input_ = dim_in
        self.output_ = dim_out          

        
def Activation(code,input_):
    """
    A class that specify the needed activation with respect to the following code conversion
        0: Relu(x)
        1: Tanh(x)
        2: Sigmoid(x)
    
    This class works as functional package of pytorch
    
    input_s:
        code        :  the code for each activation (0,1,2)
        input__tensor:  the input_ tensor
        
    returns:
        result      :  the output_ of requested activation function with the same shape as input_ tensor
    """
    
    result = torch.Tensor.new(input_.shape)
    # Relu
    if code ==0:
        result = input_ - (input_<0).float()*input_
    # Tanh
    elif code ==1:
        result = torch.tanh(input_)
    # Sig
    elif code ==2:
        result = 1.0/(1 + torch.exp(-input_))
    # error
    else: raise ValueError('Unknown Code For Activation')
    
    return result 
    
    """def Relu (self,input_):
        result = input_ - (input_<0);
        
    def Tanh (self,input_): #WARNING!!!
        result = torch.tanh(input_)
        
    def Sigmoid (self,input_):
        result = 1.0/(1 + torch.exp(-input_))

    
    def choose_activation (self, code,input_):
        if self.code ==0:
            self.result = self.Relu(input_)
        elif self.code ==1:
            self.result = self.Tanh(input_)
        elif self.code ==2:
            self.result = self.Sigmoid(input_)
        else: raise UnknownCodeForActivation
        return self.result 
    """    


# Network Class

In [173]:
class Net:
    """
    The network class. It has the following methods:
        param      :  returns the parameter which is asked for. Not the data! The object... 
                        Data is accessible through object.data method)
        make_arch  :  makes the architecture of the network by taking a sequential list of [fc1,act1,fc2,act2,...]
    """
    
    def __init__(self, seq=[]):
        self.param_list = []        # this list stores the parameters of each layer
        self.act_list = []          # this list stores the requested activation functions in codes
        #self.s_list = []
        self.make_arch(seq)         # this method makes the architecture based on the the list "seq"
        
        
    def param(self,layer):
        return self.param_list[layer]
    
    
    def make_arch(self,seq):
        seq_len = len(seq)                  # number of layer *2 (because of the activations...)
        
        for layer in range (0,seq_len,2):
            
            # seq[layer] is an instance of object "Linear". Here we get the in/out dim of the layer
            dim_in, dim_out = seq[layer].input_ , seq[layer].output_ 
            
            # initialize the weights of layer
            self.param_list.append ( Weight(dim_in, dim_out) ) 
            
            # activation recognition : encode activations in "act_list"
            if seq[layer+1]=='relu':
                self.act_list.append(0)
            elif seq[layer+1]=='tanh':
                self.act_list.append(1)
            elif seq[layer+1]=='sig':
                self.act_list.append(2)
            else: raise ValueError('Unknown Activation')
                
            
    def forward(self,x): 
        for layer, prm in enumerate(self.param_list):       # parameter = param[layer] 
            print x
            print prm.w.t()
            print prm.bias.t() 
            self.s = x.mm(prm.w.t()) + prm.bias.t()                  # written consistant for batch      
            x = Activation(self.act_list[layer],self.s)
            
        return x
        

# Draft, tests, and other stuff

In [174]:
seq = [Linear(7,2),'tanh',Linear(2,3),'relu']    
model = Net(seq)

In [175]:
x = Tensor(7).normal_()
X = torch.cat((x.view(1,-1),x.view(1,-1)*3,x.view(1,-1)*x.view(1,-1)),0)
X


-1.3873 -1.1667  1.7363 -0.4540  0.8776  0.1407 -0.0304
-4.1620 -3.5002  5.2090 -1.3621  2.6327  0.4221 -0.0913
 1.9247  1.3612  3.0149  0.2062  0.7701  0.0198  0.0009
[torch.FloatTensor of size 3x7]

In [176]:
model.forward(X)


-1.3873 -1.1667  1.7363 -0.4540  0.8776  0.1407 -0.0304
-4.1620 -3.5002  5.2090 -1.3621  2.6327  0.4221 -0.0913
 1.9247  1.3612  3.0149  0.2062  0.7701  0.0198  0.0009
[torch.FloatTensor of size 3x7]


 0.4260 -0.9387
 0.6455 -0.7858
-0.2055  0.5801
-0.0677  0.8145
-0.7658 -1.2834
-0.4100 -0.9957
 0.7761  1.8519
[torch.FloatTensor of size 7x2]


 0  0
[torch.FloatTensor of size 1x2]



TypeError: unbound method new() must be called with FloatTensor instance as first argument (got Size instance instead)

In [131]:
a.view(-1)



 1
 1
[torch.FloatTensor of size 2]

In [132]:
w1


 0.3465 -1.0615
 0.8870  0.3387
[torch.FloatTensor of size 2x2]

In [69]:
a= Tensor(b1.shape).fill_(1)

In [70]:
a


 1
 1
[torch.FloatTensor of size 2x1]

In [103]:
w1.mv(X[:,4])


1.00000e+06 *
 -1.2871
 -7.3942
[torch.FloatTensor of size 2]

In [102]:
X= Tensor(2,6).random_()


 6.8458e+06
 3.1153e+06
 9.1772e+06
[torch.FloatTensor of size 3]